In [ ]:
%%javascript
Jupyter.utils.load_extension("nb-mermaid/nb-mermaid")

<img src="bokeh-transparent.png" width="64" height="64">

# bokeh

* interaktywne wizualizacje
* nowatorskie grafiki
* renderowanie w przeglądarce webowej
* (zazwyczaj) nie ma potrzeby znania technologii webowych (HTML, JavaScript, CSS)
* integracja z jupyter notebook, numpy, pandas, matplotlib, seaborn, ...
* trzy poziomy API (`bokeh.models`, `bokeh.plotting`, `bokeh.charts`)
* wbudowany system statycznych typów danych
* możliwość tworzenia interfejsów użytkownika (`bokeh.widgets`)
* nie tylko Python (R - `rbokeh`, Julia - `bokeh.jl`, Scala - `bokeh-scala`, ...)
* możliwość rozszerzania funkcjonalność biblioteki
* serwer, strumieniowe przesyłanie danych, duże dane (próbkowanie)
* tworzone przez Continuum Analytics oraz społeczność
* licencja: BSD

## instalacja

`$ pip install bokeh`

lub

`$ conda install bokeh`

## jak to działa?

<div class="mermaid">
graph TD;
    A["Python (lub R, Julia, Scala)"]-->B["JSON + (HTML, JS, CSS)"];
    B-->C["wizualizacja w przeglądarce"];
</div>

## bokeh vs. matplotlib

* HTML/JS/CSS vs. qt/gtk + png,pdf,svg,...
* mało stabilne API vs. wysoko stabilne API
* wiele implementacji vs. tylko Python

## przykłady

* http://bokeh.pydata.org/en/latest/docs/gallery.html#gallery
* http://demo.bokehplots.com
* http://cecp.mit.edu

## kontakt

* strona: https://bokeh.pydata.org
* repozytorium: https://github.com/bokeh/bokeh
* lista mailingowa: bokeh@continuum.io

## statystyki

* github
 * 3600+ obserwatorów
 * 700+ forków
* lista mailingowa
 * 400+ członków
 * 150+ wiadomości (listopad 2015)
* liczba pobrań
 * conda: ~21k miesięcznie
 * pip: ~8k miesięcznie

### bokeh w jupyter notebook

In [ ]:
from bokeh.io import output_notebook, show
output_notebook()

### Tworzenie wykresu

In [ ]:
from numpy import cos, pi, linspace

x = linspace(-2*pi, 2*pi, 100)
y = cos(x)

In [ ]:
from bokeh.models import (Plot, DataRange1d,
    ColumnDataSource, LinearAxis, Grid)
from bokeh.models.glyphs import Circle

source = ColumnDataSource(dict(x=x, y=y))
p1 = Plot(
    x_range=DataRange1d(),
    y_range=DataRange1d(),
    title="bokeh.models")
p1.add_glyph(source, Circle(x="x", y="y",
    line_color="red", fill_color="red", fill_alpha=0.5, size=10))
xaxis = LinearAxis(axis_label="x")
yaxis = LinearAxis(axis_label="y")
p1.add_layout(xaxis, 'below')
p1.add_layout(yaxis, 'left')
p1.add_layout(Grid(dimension=0, ticker=xaxis.ticker))
p1.add_layout(Grid(dimension=1, ticker=yaxis.ticker))
p1.html

In [ ]:
from bokeh.plotting import *

p2 = figure(title="bokeh.plotting")
p2.xaxis.axis_label = "x"
p2.yaxis.axis_label = "y"
p2.circle(x, y,
    line_color="red", fill_color="red", fill_alpha=0.5, size=10)
show(p2)

In [ ]:
import pandas as pd
from bokeh.charts import Scatter
p3 = Scatter(pd.DataFrame(dict(x=x, y=y)), title="bokeh.charts")
show(p3)

In [ ]:
from bokeh.models.widgets import Tabs, Panel
t1 = Panel(child=p1, title="bokeh.models")
t2 = Panel(child=p2, title="bokeh.plotting")
t3 = Panel(child=p3, title="bokeh.charts")
tabs = Tabs(tabs=[t1, t2, t3])
show(tabs)

### Przykład: zależność spalania od mocy silnika

In [ ]:
from bokeh.sampledata.autompg import autompg

In [ ]:
autompg = autompg.copy()
autompg["liters"] = autompg.mpg.map(lambda mpg: 235.0/mpg)

In [ ]:
from bokeh.charts import Scatter
p = Scatter(autompg, x='liters', y='hp', color='cyl',
    xlabel="Spalanie (litry/100 km)", ylabel="Moc (KM)", legend='bottom_right')
show(p)

### Przykład: stopa bezrobocia w Polsce

In [ ]:
# http://stat.gov.pl/en/topics/labour-salaries/registered-unemployment/unemployment-rate-1990-2015,3,1.html
import pandas as pd
data = pd.read_csv("unemployment.csv", skipinitialspace=True)
data = data.set_index('Rok')

In [ ]:
data

In [ ]:
data.index = data.index.map(str)

In [ ]:
years = list(data.index)
months = list(data.iloc[0].index)

In [ ]:
colors = ["#75968f", "#a5bab7", "#c9d9d3",
          "#e2e2e2", "#dfccce", "#ddb7b1",
          "#cc7878", "#933b41", "#550b1d"]

In [ ]:
"""
month = []
year = []
rate = []
for y in years:
    for m in months:
        month.append(m)
        year.append(y)
        monthly_rate = data[m][y]
        rate.append(monthly_rate)
"""
        
year = sum([ [y]*12 for y in years ], [])
month = months*len(year)
rate = data.values.flatten()

In [ ]:
color = list(pd.cut(rate, len(colors), labels=colors))

In [ ]:
from bokeh.plotting import ColumnDataSource
source = ColumnDataSource(
    data=dict(month=month, year=year, rate=rate)
)

In [ ]:
from bokeh.plotting import figure

title = "Stopa bezrobocia w Polsce w latach %s - %s" % \
    (data.index.min(), data.index.max())

p = figure(
    title=title,
    x_range=list(reversed(years)),
    y_range=list(reversed(months)),
    x_axis_location="above",
    plot_width=900,
    plot_height=400,
    toolbar_location="left",
    tools="hover,tap")

In [ ]:
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "8pt"
p.axis.major_label_standoff = 0

In [ ]:
from math import pi
p.xaxis.major_label_orientation = pi/3

In [ ]:
p.rect("year", "month", 1, 1,
       source=source, color=color, line_color=None)

In [ ]:
from bokeh.models import HoverTool
p.select_one(HoverTool).tooltips = \
    "@month, @year: <b>@rate{0.00}%</b>"

In [ ]:
from bokeh.models import TapTool, CustomJS
customjs = CustomJS(
    args=dict(source=source),
    lang="coffeescript",
    code="""
Util = require "util/util"
data = source.get('data')

for i in Util.get_indices(source)
  year = data['year'][i]
  month = data['month'][i]
  rate = data['rate'][i]
  window.alert("Stopa bezrobocia na #{month}, #{year} to #{rate}%")
""")

p.select_one(TapTool).callback = customjs

In [ ]:
show(p)